In [14]:
import datetime
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import io


In [15]:
all_data_deduped=pd.read_csv('all_data_deduped.csv', sep=',', header=0)

In [16]:
user_df= all_data_deduped
movie_data = pd.read_csv("movie_titles.csv", header=None, encoding = "ISO-8859-1")
movie_data.columns = ['movie_id', 'release_year','movie_name']
netflix_data_cleaned=pd.merge(user_df, movie_data, left_on = 'movie', right_on = 'movie_id')

import scipy.io
import numpy as np

mat_path = r'C:/Users/fxi/Downloads/movieGenreBenchmark.mat'
#movieGenreData.mat'
genre_data = scipy.io.loadmat(mat_path)

movie_genre_mapping = pd.DataFrame(
    {'genre_id': genre_data['dataStructMovie'][0][0][1][:,0].tolist(),
     'movie_id':genre_data['dataStructMovie'][0][0][6][0].tolist()
    })    

k = [j[0] for j in genre_data['dataStructMovie'][0][0][8]] 

i=(list(range(1, len(k)+1)))

genre_names = pd.DataFrame(
    {'genre_id': i,
     'genre_name': k
    }) 

genre_mapping_cleaned=pd.merge(movie_genre_mapping, genre_names, left_on = 'genre_id', right_on = 'genre_id')
data=pd.merge(netflix_data_cleaned, genre_mapping_cleaned, left_on = 'movie_id', right_on = 'movie_id')

In [17]:
sorted_data=data.sort_values(['user_id', 'date'], ascending=[True, True])
sorted_data["next_user_id"]=sorted_data["user_id"].shift(-1)
sorted_data['next_date']=sorted_data['date'].shift(-1)
sorted_data['next_movie']=sorted_data['movie_name'].shift(-1)
sorted_data['next_movie_id']=sorted_data['movie_id'].shift(-1)
sorted_data['next_rating']=sorted_data['rating'].shift(-1)
sorted_data['next_genre_id']=sorted_data['genre_id'].shift(-1)
sorted_data['next_genre']=sorted_data['genre_name'].shift(-1)

#Check to see if this is the last movie we observed a user watching.
sorted_data=sorted_data.assign(same_user=sorted_data.user_id == sorted_data.next_user_id)

In [18]:
sorted_data['movie_num'] = sorted_data.sort_values(['user_id','date'], ascending=[True,True]).groupby(['user_id']).cumcount() + 1
user_actions=sorted_data.groupby('user_id')['movie_num'].max()
sorted_data_with_actions=pd.merge(sorted_data, user_actions.to_frame(), left_on = 'user_id', right_on = 'user_id')
sorted_data_with_actions.rename(columns={'movie_num_x': 'movie_num', 'movie_num_y': 'total_movies_watched'}, inplace=True)
sorted_data_with_actions.head(10)

,Unnamed: 0,user_id,rating,date,movie,movie_id,release_year,movie_name,genre_id,genre_name,next_user_id,next_date,next_movie,next_movie_id,next_rating,next_genre_id,next_genre,same_user,movie_num,total_movies_watched
0,8565167,133,5.0,2005-10-11,14691,14691,1999.0,The Matrix,12,[Sci-Fi & Fantasy],133.0,2005-12-17,Bride and Prejudice,897.0,3.0,11.0,[Romance],True,1,2
1,4637388,133,3.0,2005-12-17,897,897,2004.0,Bride and Prejudice,11,[Romance],3269.0,2005-10-01,Sleeping With the Enemy,5069.0,4.0,7.0,[Drama],False,2,2
2,3317173,3269,4.0,2005-10-01,5069,5069,1991.0,Sleeping With the Enemy,7,[Drama],3269.0,2005-12-06,Deadwood: Season 1,8197.0,5.0,13.0,[Television],True,1,2
3,21809187,3269,5.0,2005-12-06,8197,8197,2004.0,Deadwood: Season 1,13,[Television],4343.0,2004-05-12,Meet Joe Black,16465.0,4.0,7.0,[Drama],False,2,2
4,19368796,4343,4.0,2004-05-12,16465,16465,1998.0,Meet Joe Black,7,[Drama],4343.0,2005-04-01,Anchorman: The Legend of Ron Burgundy,7635.0,4.0,5.0,[Comedy],True,1,2
5,19283169,4343,4.0,2005-04-01,7635,7635,2004.0,Anchorman: The Legend of Ron Burgundy,5,[Comedy],4725.0,2005-08-11,Being John Malkovich,2122.0,4.0,5.0,[Comedy],False,2,2
6,10849813,4725,4.0,2005-08-11,2122,2122,1999.0,Being John Malkovich,5,[Comedy],4725.0,2005-09-22,The O.C.: Season 2,5204.0,4.0,13.0,[Television],True,1,2
7,4139485,4725,4.0,2005-09-22,5204,5204,2004.0,The O.C.: Season 2,13,[Television],4862.0,2005-08-19,Assault on Precinct 13,16262.0,3.0,1.0,[Action & Adventure],False,2,2
8,17762087,4862,3.0,2005-08-19,16262,16262,2005.0,Assault on Precinct 13,1,[Action & Adventure],4862.0,2005-09-06,Guess Who,5327.0,4.0,5.0,[Comedy],True,1,2
9,5057890,4862,4.0,2005-09-06,5327,5327,2005.0,Guess Who,5,[Comedy],4963.0,2005-12-30,The Amityville Horror,3689.0,1.0,9.0,[Horror],False,2,2


In [22]:
sorted_data_with_actions.to_csv('sorted_data_with_actions.csv')

In [19]:
#Delete last movie watched (next movie in the ordering is for a different user)
labeled_data=sorted_data_with_actions.loc[sorted_data_with_actions['same_user'] == True]

In [31]:
small_data = labeled_data[['user_id','genre_id','next_genre_id','next_rating']]
small_data.next_genre_id=small_data.next_genre_id.astype(int)

genre_list=list(range(1, 15))
genres=pd.DataFrame({'recommendation':genre_list})

saspr = pd.merge(small_data.assign(key=0), genres.assign(key=0), on='key').drop('key', axis=1)
saspr.loc[saspr['next_genre_id'] == saspr['recommendation'], 'fixed_rating'] = saspr['next_rating']



C:\Users\fxi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,user_id,genre_id,next_genre_id,next_rating,recommendation,fixed_rating
0,133,12,11,3.0,1,NaN
1,133,12,11,3.0,2,NaN
2,133,12,11,3.0,3,NaN
3,133,12,11,3.0,4,NaN
4,133,12,11,3.0,5,NaN
5,133,12,11,3.0,6,NaN
6,133,12,11,3.0,7,NaN
7,133,12,11,3.0,8,NaN
8,133,12,11,3.0,9,NaN
9,133,12,11,3.0,10,NaN


In [32]:
genre_reward=pd.read_csv('genre_reward.csv', sep=',', header=0)

In [39]:
saspr_r = pd.merge(saspr, genre_reward,  how='left', left_on=['recommendation','next_genre_id'], right_on = ['genre_id','next_genre_id'])

In [40]:
saspr_r.fixed_rating.fillna(saspr_r.reward, inplace=True)
saspr_r.head()

,user_id,genre_id_x,next_genre_id,next_rating,recommendation,fixed_rating,genre_id_y,reward
0,133,12,11,3.0,1,-3.0,1,-3
1,133,12,11,3.0,2,-2.0,2,-2
2,133,12,11,3.0,3,-3.0,3,-3
3,133,12,11,3.0,4,-3.0,4,-3
4,133,12,11,3.0,5,-3.0,5,-3


In [43]:
final_labeled_data_small= saspr_r[['genre_id_x','recommendation','next_genre_id','fixed_rating']]
final_labeled_data_small.rename(columns={'genre_id_x': 's','recommendation': 'a','next_genre_id': 'sp','fixed_rating': 'r'}, inplace=True)
final_labeled_data_small.head(10)

C:\Users\fxi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,s,a,sp,r
0,12,1,11,-3.0
1,12,2,11,-2.0
2,12,3,11,-3.0
3,12,4,11,-3.0
4,12,5,11,-3.0
5,12,6,11,-5.0
6,12,7,11,-2.0
7,12,8,11,-3.0
8,12,9,11,-5.0
9,12,10,11,-2.0


In [45]:
final_labeled_data_small.to_csv('final_labeled_data_v2.csv')

#np.random.seed(123)

#msk = np.random.rand(len(labeled_data)) < 0.8

#train = labeled_data[msk]

#test = labeled_data[~msk]